In [3]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install pypdf

   ---------------------------------------- 0.0/302.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/302.0 kB ? eta -:--:--
   ------------------------------------ --- 276.5/302.0 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 302.0/302.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
pip install sentence-transformers

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.0 kB 682.7 kB/s eta 0:00:01
     -------------------------------------- 44.0/44.0 kB 718.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.2 MB/s eta 0:00:00
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.1 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install chromadb


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Leer el documentos

In [4]:
def cargar_documentos(ruta_archivo):
    """Carga documentos en PDF, TXT o DOCX y los convierte en texto."""
    if ruta_archivo.endswith(".pdf"):
        loader = PyPDFLoader(ruta_archivo)
    elif ruta_archivo.endswith(".txt"):
        loader = TextLoader(ruta_archivo)
    elif ruta_archivo.endswith(".docx"):
        loader = Docx2txtLoader(ruta_archivo)
    else:
        raise ValueError("Formato no soportado. Usa PDF, TXT o DOCX.")
    
    documentos = loader.load()
    
    return documentos

In [12]:
# Leer documento pdf
loader = PyPDFLoader("data/1-01-Curso_PLN.pdf")
pages = loader.load()
pages

[Document(metadata={'producer': 'Acrobat Distiller 19.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-10-30T15:27:39-05:00', 'author': 'USUARIO', 'moddate': '2024-10-30T15:27:39-05:00', 'title': 'Microsoft PowerPoint - 1-01-Curso_PLN', 'source': 'data/1-01-Curso_PLN.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='30/10/2024\n3011176 - Procesamiento del Lenguaje \nNatural\nFACULTAD DE MINAS\nSede Medellín\nSINTELWEB\nGrupo de Investigación\nSistemas Inteligentes Web\nJAIME ALBERTO GUZMAN LUNA, Ph.D\nInformación de contacto\n\uf06e Jaime Alberto Guzmán Luna\n\uf06e Doctor en Ingeniería de Sistemas e Informática\n\uf06e M.Sc. en Ingeniería de Sistemas e Informática\n\uf06e Especialista en comunicación educativa\n\uf06e Ingeniero Civil\n\uf06e Áreas de trabajo\n\uf06e Inteligencia Artificial\n\uf06e Sistemas de recuperación de información\n\uf06e Planificación automática de procesos \n\uf06e Aprendizaje de máquinas\n\uf06e PLN con aprendiza

# Split document

In [6]:
def split_text(text):
    """Divide el texto en oraciones."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, # Número de caracteres por chunk o fragmento
                                              chunk_overlap=50, # Número de caracteres de traslape entre chunks
                                              length_function=len, # Función para obtener la longitud de un texto
                                              separators=["\n\n", "\n", " "], # Separadores de palabras
                                              )
    textos_fragmentados = text_splitter.split_documents(text)
    
    return textos_fragmentados

In [13]:
chunck=split_text(pages)
chunck

[Document(metadata={'producer': 'Acrobat Distiller 19.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-10-30T15:27:39-05:00', 'author': 'USUARIO', 'moddate': '2024-10-30T15:27:39-05:00', 'title': 'Microsoft PowerPoint - 1-01-Curso_PLN', 'source': 'data/1-01-Curso_PLN.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='30/10/2024\n3011176 - Procesamiento del Lenguaje \nNatural\nFACULTAD DE MINAS\nSede Medellín\nSINTELWEB\nGrupo de Investigación\nSistemas Inteligentes Web\nJAIME ALBERTO GUZMAN LUNA, Ph.D\nInformación de contacto\n\uf06e Jaime Alberto Guzmán Luna\n\uf06e Doctor en Ingeniería de Sistemas e Informática\n\uf06e M.Sc. en Ingeniería de Sistemas e Informática\n\uf06e Especialista en comunicación educativa\n\uf06e Ingeniero Civil\n\uf06e Áreas de trabajo\n\uf06e Inteligencia Artificial\n\uf06e Sistemas de recuperación de información'),
 Document(metadata={'producer': 'Acrobat Distiller 19.0 (Windows)', 'creator': 'PScript5.dll Version

# Crear embeddings

Se hará uso de un modelo de Hugging Face all-MiniLM-L6-v2

In [7]:
# Usar modelo de Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\HP\AppData\Local\Temp\ipykernel_15376\42392456.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\HP\Documents\Trabajo-4-RN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Almacenar  embeddings en ChromaDB

In [20]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Lista de valores unicos para documentos
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    #Crea una database de chroma
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [21]:
vectorstore = create_vectorstore(chunks=chunck, 
                                 embedding_function=embeddings, 
                                 vectorstore_path="./vectorstore")

C:\Users\HP\AppData\Local\Temp\ipykernel_15376\3124993982.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# Consulta de datos relevantes

In [ ]:
#Cargamos el vectorstore
database = Chroma(persist_directory="vectorstore",embedding_function=embeddings)

C:\Users\HP\AppData\Local\Temp\ipykernel_15376\1726024856.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  database = Chroma(persist_directory="vectorstore",embedding_function=embeddings)


In [28]:
from langchain_core.prompts import ChatPromptTemplate

#Hacemos el prompt configurado con el retrieve
PROMPT_TEMPLATE = """
Eres un asistente para la generación de materiales educativos basados en un programa de curso.
Utiliza la siguiente información recuperada para crear materiales de aprendizaje estructurados.

---

**Título del curso:** {title}

**Temas principales:** {topics}

**Objetivos de aprendizaje:** {objectives}

**Lecturas y recursos recomendados:** {resources}

**Preguntas para discusión:** {discussion_questions}

**Ejercicios y problemas de práctica:** {practice_problems}

---

Basándote en la información anterior, genera los siguientes materiales educativos:
1. Notas detalladas de clase.
2. Problemas de práctica con soluciones.
3. Preguntas para discusión.
4. Objetivos de aprendizaje específicos para cada tema.
5. Lecturas y recursos sugeridos.

Estructura la respuesta de manera clara y detallada. Si no encuentras algo en el documento puedes utilizar información de internet
"""

#Llamamos al retriever
retriever = vectorstore.as_retriever(search_type="similarity")

title = retriever.invoke("¿Cuál es el título del curso?")
topics = retriever.invoke("¿Cuáles son los temas principales que se cubren en el curso?")
objectives = retriever.invoke("¿Cuáles son los objetivos de aprendizaje de este curso?")
resources = retriever.invoke("¿Cuáles son las lecturas o recursos recomendados para este curso?")
discussion_questions = retriever.invoke("¿Cuáles son algunas preguntas para discusión en este curso?")
practice_problems = retriever.invoke("¿Qué ejercicios o problemas de práctica se incluyen en el programa del curso?")

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
#Insertamos los valores en el prompt
final_prompt = PROMPT_TEMPLATE.format(
    title=title,
    topics=topics,
    objectives=objectives,
    resources=resources,
    discussion_questions=discussion_questions,
    practice_problems=practice_problems
)

print(final_prompt)


Eres un asistente para la generación de materiales educativos basados en un programa de curso.
Utiliza la siguiente información recuperada para crear materiales de aprendizaje estructurados.

---

**Título del curso:** [Document(metadata={'author': 'USUARIO', 'creationdate': '2024-10-30T15:27:39-05:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2024-10-30T15:27:39-05:00', 'page': 1, 'page_label': '2', 'producer': 'Acrobat Distiller 19.0 (Windows)', 'source': 'data/1-01-Curso_PLN.pdf', 'title': 'Microsoft PowerPoint - 1-01-Curso_PLN', 'total_pages': 4}, page_content='\uf06e Aplicar modelos y herramientas de PLN avanzadas a problemas del mundo real\n(traducción automática, Preguntas&Respuestas- QA, análisis de conversaciones,\netc)\nContenido del Curso (1)\n\uf06e TEMA 1: Introducción al Procesamiento de Lenguaje Natural.\n\uf06e Generalidades del Procesamiento de Lenguaje Natural y su importancia.\n\uf06e Limpieza y normalización de textos en Python\n\uf06e Manejo de errores

# Definimos el LLM

In [ ]:
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
